In [ ]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import openai
import time
from fpdf import FPDF

openai.api_key = ''

## Load the dataset

In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/rpdpscl/AI_First_Day4_PatPascual/refs/heads/main/shoprisk_lazrisk_data.csv')

In [6]:
df.sample(5)

,Date,Platform,Courier Name,Region/City,Number of Orders,Delayed Deliveries,Failed Deliveries,Returns,Average Delivery Time (Days),Total Order Value (PHP),...,Courier Reliability Score,Courier Capacity Utilization,Courier Experience Years,Courier Staff Turnover Rate,Courier Vehicle Condition Score,Fuel Price (PHP),Competitor Delivery Rate (PHP),Market Demand Index,Seasonal Factor,Holiday Period Flag
34,2023-05-05,Lazada,CourierZ,Bacolod,112,12,3,5,3.1,112000,...,7.4,79.09,1,20.43,9,54.09,111.85,8,10,0
25,2023-04-26,Lazada,CourierX,Cagayan de Oro,112,11,4,4,3.0,112000,...,7.0,76.91,3,14.34,8,52.31,96.99,1,9,0
74,2023-06-14,Shopee,CourierX,Davao,128,7,6,8,2.7,128000,...,6.7,94.21,6,24.67,5,58.67,82.69,10,1,1
20,2023-04-21,Lazada,CourierX,Baguio,95,9,5,8,2.8,95000,...,9.9,71.64,5,10.33,9,60.00,90.84,3,4,0
13,2023-04-14,Shopee,CourierX,Baguio,104,7,4,4,3.4,104000,...,9.9,82.76,4,18.14,10,61.14,106.17,3,10,0


## Understanding the Dataset

In [7]:
# Get Column Names and Data Types
column_info = df.dtypes
print(column_info)

Date                                object
Platform                            object
Courier Name                        object
Region/City                         object
Number of Orders                     int64
Delayed Deliveries                   int64
Failed Deliveries                    int64
Returns                              int64
Average Delivery Time (Days)       float64
Total Order Value (PHP)              int64
Courier Fees (PHP)                 float64
Estimated Financial Loss (PHP)     float64
Product Category                    object
Customer Satisfaction Score        float64
Delivery Success Rate              float64
Peak Season Flag                     int64
Weather Condition                   object
Delivery Attempt Count               int64
Last Mile Distance (km)            float64
Compensation Claims (PHP)          float64
Storage Cost (PHP)                 float64
Redelivery Cost (PHP)              float64
Insurance Claims (PHP)             float64
Refund Proc

In [8]:
# Identify Categorical and Numerical Features
categorical_features = df.select_dtypes(include=['object']).columns
numerical_features = df.select_dtypes(include=['int64', 'float64']).columns

print("categorical_features:", categorical_features)
print("numerical_features:", numerical_features)



categorical_features: Index(['Date', 'Platform', 'Courier Name', 'Region/City', 'Product Category',
       'Weather Condition', 'Delivery Zone Type', 'Flood Risk Zone',
       'Preferred Delivery Time', 'Package Size Category'],
      dtype='object')
numerical_features: Index(['Number of Orders', 'Delayed Deliveries', 'Failed Deliveries',
       'Returns', 'Average Delivery Time (Days)', 'Total Order Value (PHP)',
       'Courier Fees (PHP)', 'Estimated Financial Loss (PHP)',
       'Customer Satisfaction Score', 'Delivery Success Rate',
       'Peak Season Flag', 'Delivery Attempt Count', 'Last Mile Distance (km)',
       'Compensation Claims (PHP)', 'Storage Cost (PHP)',
       'Redelivery Cost (PHP)', 'Insurance Claims (PHP)',
       'Refund Processing Fee (PHP)', 'Traffic Congestion Index',
       'Accessibility Score', 'Hub Distance (km)',
       'Customer Availability Rate', 'Address Accuracy Score',
       'Customer Order Frequency', 'Previous Return Count',
       'Item Fragili

In [11]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display =10):
    uniques = df[col].unique().tolist()
    if len(uniques) > max_display:
        return uniques[:max_display] + ['...']
    return uniques

unique_values = {col: get_unique_values(col) for col in categorical_features}


print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques}")
print()

Unique Values for Categorical Features:
 - Date: ['2023-04-01', '2023-04-02', '2023-04-03', '2023-04-04', '2023-04-05', '2023-04-06', '2023-04-07', '2023-04-08', '2023-04-09', '2023-04-10', '...']
 - Platform: ['Shopee', 'Lazada']
 - Courier Name: ['CourierZ', 'CourierY', 'CourierX']
 - Region/City: ['Cagayan de Oro', 'Manila', 'Iloilo', 'Bacolod', 'General Santos', 'Cebu', 'Baguio', 'Quezon City', 'Davao']
 - Product Category: ['Automotive', 'Fashion', 'Toys', 'Health & Beauty', 'Groceries', 'Electronics', 'Books', 'Home Appliances', 'Sports']
 - Weather Condition: ['typhoon', 'heavy_rain', 'normal']
 - Delivery Zone Type: ['suburban', 'urban', 'rural']
 - Flood Risk Zone: ['medium', 'high', 'low']
 - Preferred Delivery Time: ['afternoon', 'morning', 'evening']
 - Package Size Category: ['large', 'small', 'medium']



In [16]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_features].describe()
print("Descriptive Statistics for Numerical Features:\n", numerical_stats, "\n")

Descriptive Statistics for Numerical Features:
        Number of Orders  Delayed Deliveries  Failed Deliveries     Returns  \
count        100.000000          100.000000         100.000000  100.000000   
mean         108.060000            9.040000           4.510000    6.000000   
std           13.638048            2.073985           1.210184    1.483921   
min           81.000000            6.000000           3.000000    4.000000   
25%           99.750000            7.000000           3.000000    5.000000   
50%          109.500000            9.000000           4.000000    6.000000   
75%          117.500000           11.000000           6.000000    7.000000   
max          130.000000           12.000000           6.000000    8.000000   

       Average Delivery Time (Days)  Total Order Value (PHP)  \
count                     100.00000               100.000000   
mean                        3.15400            108060.000000   
std                         0.26378             13638.048

## Generating the Template

In [21]:
def generate_template(df,
                      column_info,
                      categorical_features,
                      numerical_features,
                      unique_values,
                      numerical_stats):
    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f" - {col}: {dtype}\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, uniques in unique_values.items():
        unique_values_str += f" - {col}: {uniques}\n"

    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics for Numerical Features:\n"
    for col in numerical_features:
        numerical_stats_str += f" - {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"   - {stat_name}: {value}\n"

    # Define the system prompt
    system_prompt = """
    You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness.
    """

    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."

    Output only the template without any explanation or introduction.
    The template's variables will be dynamically replaced so make sure they're formatted properly.
    """

    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None

In [22]:
# Generate the template
template = generate_template(df,
                             column_info,
                             categorical_features,
                             numerical_features,
                             unique_values,
                             numerical_stats)
print(template)

"On {Date}, the delivery platform {Platform} utilized the services of {Courier Name} to fulfill {Number of Orders} orders in the {Region/City} region. Out of these, {Delayed Deliveries} deliveries were delayed, {Failed Deliveries} failed to reach their destination, and {Returns} were processed. The average delivery time was {Average Delivery Time (Days)} days, contributing to a total order value of {Total Order Value (PHP)} PHP. Courier fees amounted to {Courier Fees (PHP)} PHP, leading to an estimated financial loss of {Estimated Financial Loss (PHP)} PHP. The product category for these orders included {Product Category}, and customer satisfaction was reflected in a score of {Customer Satisfaction Score}. The delivery success rate stood at {Delivery Success Rate}%, with a peak season flag of {Peak Season Flag}. Weather conditions were reported as {Weather Condition}, and the delivery attempt count was {Delivery Attempt Count}, covering a last mile distance of {Last Mile Distance (km)}

## Populating the Template with Actual Values

In [26]:
def populate_template(template, row):

    # Convert row to dictionary and replace NaN with 'N/A'
    row_dict = row.to_dict()
    for key, value in row_dict.items():
        if pd.isna(value):
            row_dict[key] = 'N/A'
    paragraph = template.format(**row_dict)
    return paragraph

    # Generate the populated template per row
    paragraph = template.format(**row_dict)
    return paragraph


In [27]:
# Apply the populate_template function to each row in the dataframe
df['paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)


In [28]:
df['paragraph'][0]

'"On 2023-04-01, the delivery platform Shopee utilized the services of CourierZ to fulfill 112 orders in the Cagayan de Oro region. Out of these, 7 deliveries were delayed, 4 failed to reach their destination, and 5 were processed. The average delivery time was 2.8 days, contributing to a total order value of 112000 PHP. Courier fees amounted to 11200.0 PHP, leading to an estimated financial loss of 1400.0 PHP. The product category for these orders included Automotive, and customer satisfaction was reflected in a score of 4.5. The delivery success rate stood at 95.59%, with a peak season flag of 0. Weather conditions were reported as typhoon, and the delivery attempt count was 3, covering a last mile distance of 4.09 km. Compensation claims reached 976.14 PHP, while storage and redelivery costs were 125.37 PHP and 401.62 PHP respectively. Insurance claims totaled 1238.98 PHP, and refund processing fees were 91.12 PHP. The delivery zone type was categorized as suburban, with a flood ris

In [29]:
df.sample(5)

,Date,Platform,Courier Name,Region/City,Number of Orders,Delayed Deliveries,Failed Deliveries,Returns,Average Delivery Time (Days),Total Order Value (PHP),...,Courier Capacity Utilization,Courier Experience Years,Courier Staff Turnover Rate,Courier Vehicle Condition Score,Fuel Price (PHP),Competitor Delivery Rate (PHP),Market Demand Index,Seasonal Factor,Holiday Period Flag,paragraph
37,2023-05-08,Shopee,CourierZ,Cebu,130,6,4,7,3.1,130000,...,94.50,1,21.62,8,50.36,90.56,5,4,0,"""On 2023-05-08, the delivery platform Shopee u..."
59,2023-05-30,Shopee,CourierY,Iloilo,83,9,4,7,3.5,83000,...,92.77,7,13.90,9,51.51,115.17,7,1,0,"""On 2023-05-30, the delivery platform Shopee u..."
9,2023-04-10,Lazada,CourierY,Iloilo,114,7,5,5,2.8,114000,...,74.55,3,19.09,8,54.68,110.00,2,3,0,"""On 2023-04-10, the delivery platform Lazada u..."
8,2023-04-09,Shopee,CourierZ,Cagayan de Oro,99,10,3,5,3.0,99000,...,89.64,3,14.33,10,57.82,115.62,3,10,0,"""On 2023-04-09, the delivery platform Shopee u..."
98,2023-07-08,Lazada,CourierX,Cebu,106,7,6,8,3.1,106000,...,94.51,7,15.48,6,65.64,113.48,7,1,0,"""On 2023-07-08, the delivery platform Lazada u..."


## Saving the Populated Templates (Natural Language Texts)

In [32]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")


In [31]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [34]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['paragraph'].to_list(),
                    txt_filename='nlg.txt')

save_content_to_pdf(df['paragraph'].to_list(),
                    pdf_filename='nlg.pdf')

All content has been saved as 'nlg.txt'.

All content has been saved as 'nlg.pdf'.

